<a href="https://colab.research.google.com/github/PriyanshuGupta1/GeneratingMusicUsingLSTM/blob/master/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import tensorflow.keras as keras
from keras.applications.densenet import layers
import numpy as np

mapping_path="/content/mapping.json"
sequence_length=64
single_file_dataset = "/content/file_dataset" 
LOSS = "sparse_categorical_crossentropy"    # error function for training
LEARNING_RATE = 0.001
EPOCHS = 50
NUM_UNITS = [256]   #number of units of neuron in internal layer of network this is one layer with 256 neurons
BATCH_SIZE = 64     # amount of samples it will see before running back propogation
save_model_path = "model.h5"   # .h5 for saving keras model


def convert_songs_to_int(songs):
    # covert song in symbolic representaition to integer into sequences so that we can train our LSTM model
    # takes input as songs which is string of integer
    int_songs = []
    # store all dataset and mapping
    # load the mappings
    with open(mapping_path, "r") as fp:
        mappings = json.load(fp)
        # dictionary which contain all our symbols

    # cast songs string to list
    songs = songs.split()
    # splits a string at empty spaces and create items in list

    # map song to int
    for symbol in songs:
        int_songs.append(mappings[symbol])
    #print(int_songs)
    return int_songs


def generating_training_sequences(sequence_length):
    # load the songs and map to int
    songs = load(single_file_dataset)
    #print(songs)
    int_songs = convert_songs_to_int(songs)
    # genearte the training sequence
    num_sequences=len(int_songs)-sequence_length

    input=[]
    target=[]

    for i in range(num_sequences):
        input.append(int_songs[i:i+sequence_length])
        target.append(int_songs[i+sequence_length])
    """ one-hot encode the sequences
    inputs: (# of sequences ,sequence length,vocabulary length)
    easiest way to work with categorical data for neural network"""
    
    vocabulary_size=len(set(int_songs))
    inputs = keras.utils.to_categorical(input, num_classes=vocabulary_size)
    # input will now be a 3D array
    targets=np.array(target)
    
    return inputs,targets

def output_units_(mapping_path):
    with open(mapping_path, "r") as fp:
        mappings = json.load(fp)
    return len(mappings.keys())


def build_model(output_units, num_units, loss, learning_rate):
    # create model architecture
    input = keras.layers.Input(shape=(None, output_units))
    # shape is of the data which we will input in our model
    # None enables us to have as many timestamps as we want ,we than can generate however melodies we want
    # output_units tells us how many elements we have for each timestamp i.e our vocabulary size
    x = keras.layers.LSTM(num_units[0])(input)
    # add another node to graph

    x = keras.layers.Dropout(0.2)(x)
    # to avoid overfitting we use dropout layer
    # here the rate refers to amount of input data to be dropped
    output = keras.layers.Dense(output_units, activation="softmax")(x)

    model = keras.Model(input, output)

    # compile model

    model.compile(loss=loss, optimizer=keras.optimizers.Adam(lr=learning_rate), metrics=["accuracy"])
    model.summary()
    # print some information about all the model
    return model


def train(output_units=output_units_(mapping_path), num_units=NUM_UNITS, loss=LOSS, learning_rate=LEARNING_RATE):
    # it will do all the functions that will train our dataset
    # generate training sequences
    # it is already created in preprocess.py

    input, target = generating_training_sequences(sequence_length)
    # build the network

    model = build_model(output_units, num_units, loss, learning_rate)
    # output units i.e how many neurons are there in output layer

    # train the model
    model.fit(input, target, epochs=EPOCHS, batch_size=BATCH_SIZE)
    # save the model for future uses

    model.save(save_model_path)


if __name__ == '__main__':
    train()


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None, 38)]        0         
                                                                 
 lstm (LSTM)                 (None, 256)               302080    
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense (Dense)               (None, 38)                9766      
                                                                 
Total params: 311,846
Trainable params: 311,846
Non-trainable params: 0
_________________________________________________________________
Epoch 1/50


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


5663/5663 [==============================] - 1477s 260ms/step - loss: 0.6864 - accuracy: 0.7881
Epoch 2/50
5663/5663 [==============================] - 1485s 262ms/step - loss: 0.5754 - accuracy: 0.8141
Epoch 3/50
5663/5663 [==============================] - 1482s 262ms/step - loss: 0.5393 - accuracy: 0.8266
Epoch 4/50
5663/5663 [==============================] - 1460s 258ms/step - loss: 0.5127 - accuracy: 0.8339
Epoch 5/50
5663/5663 [==============================] - 1445s 255ms/step - loss: 0.4949 - accuracy: 0.8392
Epoch 6/50
5663/5663 [==============================] - 1455s 257ms/step - loss: 0.4804 - accuracy: 0.8433
Epoch 7/50
5663/5663 [==============================] - 1491s 263ms/step - loss: 0.4662 - accuracy: 0.8470
Epoch 8/50
5663/5663 [==============================] - 1479s 261ms/step - loss: 0.4531 - accuracy: 0.8514
Epoch 9/50
5663/5663 [==============================] - 1468s 259ms/step - loss: 0.4422 - accuracy: 0.8546
Epoch 10/50
5663/5663 [=========================